# RULA Human Analysis

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import scipy.stats as stats

#file_path = 'AAHF21-RW-NO-VR-TRIAL-3-MOTIVE.csv'
file_path = 'ATSF08-RW-WITH-VR-TRIAL-1-MOTIVE.csv'
#file_path = 'MGNH15-RW-NO-VR-TRIAL-2-MOTIVE.csv'
data = pd.read_csv(file_path,on_bad_lines='skip',sep=',',header=[1,2,3,4,5])
data.columns = data.columns.droplevel(2)
data.columns = ['_'.join(map(str, col)).strip() for col in data.columns.values]
data.rename(columns={'Type_Name_Unnamed: 1_level_3_Time (Seconds)':'Time'},inplace=True)
data.rename(columns={'Unnamed: 0_level_0_Unnamed: 0_level_1_Unnamed: 0_level_3_Frame':'Frame'},inplace=True)

In [ ]:
# Utility function to calculate angles
def calculate_angle(a, b, c):
    """
    Calculate the angle between three points a, b, and c
    with b as the vertex.
    """
    ab = a - b
    cb = c - b
    dot_product = np.dot(ab, cb)
    magnitude = np.linalg.norm(ab) * np.linalg.norm(cb)
    angle = np.arccos(dot_product / magnitude)  # Angle in radians
    return np.degrees(angle)  # Convert to degrees

# Calculate joint angles
def calculate_joint_angles(row):
    # Upper Arm angle (T10 -> LSHO -> LELB)
    upper_arm_angle = calculate_angle(
        np.array([row['Marker_Skeleton:T10_Position_X'], row['Marker_Skeleton:T10_Position_Y'], row['Marker_Skeleton:T10_Position_Z']]),
        np.array([row['Marker_Skeleton:LSHO_Position_X'], row['Marker_Skeleton:LSHO_Position_Y'], row['Marker_Skeleton:LSHO_Position_Z']]),
        np.array([row['Marker_Skeleton:LELB_Position_X'], row['Marker_Skeleton:LELB_Position_Y'], row['Marker_Skeleton:LELB_Position_Z']])
    )
    
    # Lower Arm angle (LSHO -> LELB -> LWRB)
    lower_arm_angle = calculate_angle(
        np.array([row['Marker_Skeleton:LSHO_Position_X'], row['Marker_Skeleton:LSHO_Position_Y'], row['Marker_Skeleton:LSHO_Position_Z']]),
        np.array([row['Marker_Skeleton:LELB_Position_X'], row['Marker_Skeleton:LELB_Position_Y'], row['Marker_Skeleton:LELB_Position_Z']]),
        np.array([row['Marker_Skeleton:LWRB_Position_X'], row['Marker_Skeleton:LWRB_Position_Y'], row['Marker_Skeleton:LWRB_Position_Z']])
    )

    # Wrist angle (LFRM -> LWRB -> LFIN)
    wrist_angle = calculate_angle(
        np.array([row['Marker_Skeleton:LFRM_Position_X'], row['Marker_Skeleton:LFRM_Position_Y'], row['Marker_Skeleton:LFRM_Position_Z']]),
        np.array([row['Marker_Skeleton:LWRB_Position_X'], row['Marker_Skeleton:LWRB_Position_Y'], row['Marker_Skeleton:LWRB_Position_Z']]),
        np.array([row['Marker_Skeleton:LFIN_Position_X'], row['Marker_Skeleton:LFIN_Position_Y'], row['Marker_Skeleton:LFIN_Position_Z']])
    )
    wrist_angle = wrist_angle - 90
    

    # Neck angle (LBHD -> C7 -> T10)
    neck_angle = calculate_angle(
        np.array([row['Marker_Skeleton:LBHD_Position_X'], row['Marker_Skeleton:LBHD_Position_Y'], row['Marker_Skeleton:LBHD_Position_Z']]),
        np.array([row['Marker_Skeleton:C7_Position_X'], row['Marker_Skeleton:C7_Position_Y'], row['Marker_Skeleton:C7_Position_Z']]),
        np.array([row['Marker_Skeleton:T10_Position_X'], row['Marker_Skeleton:T10_Position_Y'], row['Marker_Skeleton:T10_Position_Z']])
    )
    neck_angle = 180 - neck_angle 
    
    if 'Marker_Skeleton:LPSI_Position_X' in row:
            
        # Trunk angle (LPSI -> T10 -> C7)
        trunk_angle = calculate_angle(
            np.array([row['Marker_Skeleton:LPSI_Position_X'], row['Marker_Skeleton:LPSI_Position_Y'], row['Marker_Skeleton:LPSI_Position_Z']]),
            np.array([row['Marker_Skeleton:T10_Position_X'], row['Marker_Skeleton:T10_Position_Y'], row['Marker_Skeleton:T10_Position_Z']]),
            np.array([row['Marker_Skeleton:C7_Position_X'], row['Marker_Skeleton:C7_Position_Y'], row['Marker_Skeleton:C7_Position_Z']])
        )
    else:
        # Trunk angle (RPSI -> T10 -> C7)
        trunk_angle = calculate_angle(
            np.array([row['Marker_Skeleton:RPSI_Position_X'], row['Marker_Skeleton:RPSI_Position_Y'], row['Marker_Skeleton:RPSI_Position_Z']]),
            np.array([row['Marker_Skeleton:T10_Position_X'], row['Marker_Skeleton:T10_Position_Y'], row['Marker_Skeleton:T10_Position_Z']]),
            np.array([row['Marker_Skeleton:C7_Position_X'], row['Marker_Skeleton:C7_Position_Y'], row['Marker_Skeleton:C7_Position_Z']])
        )
    trunk_angle = 180 - trunk_angle

    return pd.Series({
        'Upper_Arm_Angle': upper_arm_angle,
        'Lower_Arm_Angle': lower_arm_angle,
        'Wrist_Angle': wrist_angle,
        'Neck_Angle': neck_angle,
        'Trunk_Angle': trunk_angle
    })

# Apply the function to calculate joint angles
angles_df = data.apply(calculate_joint_angles, axis=1)

In [ ]:

plt.figure(1,figsize=(15, 3))
plt.plot(data['Time'], angles_df['Upper_Arm_Angle'], linestyle='-', color='b', label='Upper Arm Angle')
plt.xlabel('Time (s)')
plt.ylabel('Upper Arm Angle (degree)')
plt.title('Upper Arm Angle vs Time')
plt.legend()
plt.grid(True)
#plt.show()

plt.figure(2,figsize=(15, 3))
plt.plot(data['Time'], angles_df['Lower_Arm_Angle'], linestyle='-', color='b', label='Lower Arm Angle')
plt.xlabel('Time (s)')
plt.ylabel('Lower Arm Angle (degree)')
plt.title('Lower Arm Angle vs Time')
plt.legend()
plt.grid(True)

plt.figure(3,figsize=(15, 3))
plt.plot(data['Time'], angles_df['Wrist_Angle'], linestyle='-', color='b', label='Wrist Angle')
plt.xlabel('Time (s)')
plt.ylabel('Wrist Angle (degree)')
plt.title('Wrist vs Time')
plt.legend()
plt.grid(True)

plt.figure(4,figsize=(15, 3))
plt.plot(data['Time'], angles_df['Neck_Angle'], linestyle='-', color='b', label='Neck Angle')
plt.xlabel('Time (s)')
plt.ylabel('Neck Angle (degree)')
plt.title('Neck Angle vs Time')
plt.legend()
plt.grid(True)

plt.figure(5,figsize=(15, 3))
plt.plot(data['Time'], angles_df['Trunk_Angle'], linestyle='-', color='b', label='Trunk Angle')
plt.xlabel('Time (s)')
plt.ylabel('Trunk Angle (degree)')
plt.title('Trunk Angle vs Time')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
def clean_sudden_changes(dataframe, columns, threshold):
    """
    Removes sudden changes in specified columns of a DataFrame by replacing them
    with the preceding values.

    :param dataframe: pd.DataFrame - The input DataFrame
    :param columns: list - List of column names to clean
    :param threshold: float - The allowed threshold for change
    :return: pd.DataFrame - The modified DataFrame
    """
    # Copy the dataframe to avoid modifying the original
    cleaned_df = dataframe.copy()
    
    # Iterate over the specified columns
    for column in columns:
        # Iterate over the rows for the current column
        for i in range(1, len(cleaned_df)):
            current_value = cleaned_df.loc[i, column]
            previous_value = cleaned_df.loc[i - 1, column]
            
            # Check for sudden change
            if abs(current_value - previous_value) > threshold:
                cleaned_df.loc[i, column] = previous_value
    
    return cleaned_df

# Example usage
columns_to_clean = ['Upper_Arm_Angle', 'Lower_Arm_Angle','Wrist_Angle','Neck_Angle','Trunk_Angle']  # List of columns to clean
threshold = 15  # Define a threshold for sudden changes
angles_df_filtered = clean_sudden_changes(angles_df, columns_to_clean, threshold)



plt.figure(1,figsize=(15, 3))
plt.plot(data['Time'], angles_df_filtered['Upper_Arm_Angle'], linestyle='-', color='b', label='Upper Arm Angle')
plt.xlabel('Time (s)')
plt.ylabel('Upper Arm Angle (degree)')
plt.title('Upper Arm Angle - ATSF08 with VR, Trial 3')
plt.legend()
plt.grid(True)
#plt.show()

plt.figure(2,figsize=(15, 3))
plt.plot(data['Time'], angles_df_filtered['Lower_Arm_Angle'], linestyle='-', color='b', label='Lower Arm Angle')
plt.xlabel('Time (s)')
plt.ylabel('Lower Arm Angle (degree)')
plt.title('Lower Arm Angle - ATSF08 with VR, Trial 3')
plt.legend()
plt.grid(True)

plt.figure(3,figsize=(15, 3))
plt.plot(data['Time'], angles_df_filtered['Wrist_Angle'], linestyle='-', color='b', label='Wrist Angle')
plt.xlabel('Time (s)')
plt.ylabel('Wrist Angle (degree)')
plt.title('Wrist Angle - ATSF08 with VR, Trial 3')
plt.legend()
plt.grid(True)

plt.figure(4,figsize=(15, 3))
plt.plot(data['Time'], angles_df_filtered['Neck_Angle'], linestyle='-', color='b', label='Neck Angle')
plt.xlabel('Time (s)')
plt.ylabel('Neck Angle (degree)')
plt.title('Neck Angle - ATSF08 with VR, Trial 3')
plt.legend()
plt.grid(True)

plt.figure(5,figsize=(7.5, 2))
plt.plot(data['Time'], angles_df_filtered['Trunk_Angle'], linestyle='-', color='b', label='Trunk Angle')
plt.xlabel('Time (s)')
plt.ylabel('Trunk Angle (degree)')
plt.title('Trunk Angle - ATSF08 with VR, Trial 3')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
# Create a figure
fig, ax = plt.subplots(figsize=(9, 5))

# Plot all angles on the same graph with different colors
ax.plot(data['Time'], angles_df_filtered['Upper_Arm_Angle'], linestyle='-', color='b', label='Upper Arm Angle')
ax.plot(data['Time'], angles_df_filtered['Lower_Arm_Angle'], linestyle='-', color='r', label='Lower Arm Angle')
ax.plot(data['Time'], angles_df_filtered['Wrist_Angle'], linestyle='-', color='g', label='Wrist Angle')
ax.plot(data['Time'], angles_df_filtered['Neck_Angle'], linestyle='-', color='m', label='Neck Angle')
ax.plot(data['Time'], angles_df_filtered['Trunk_Angle'], linestyle='-', color='c', label='Trunk Angle')

# Labels and title
ax.set_xlabel('Time (s)')
ax.set_ylabel('Angle (degree)')
ax.set_title('Angles Over Time - ATSF08 with VR, Trial 3')
ax.grid(True)
ax.legend()

plt.tight_layout()
output_path = "Rula_angles.pdf"  # Specify your desired file path
plt.savefig(output_path, format='pdf')

# Show the plot
plt.show()


In [ ]:
# RULA scoring based on angles (simplified example)
def rula_scoring(row):
    """
    Computes RULA scores based on the given joint angles.
    """

    # Upper Arm Score
    if row['Upper_Arm_Angle'] > 90:
        upper_arm_score = 4
    elif row['Upper_Arm_Angle'] >= 45:
        upper_arm_score = 3
    elif row['Upper_Arm_Angle'] >= 20:
        upper_arm_score = 2
    elif row['Upper_Arm_Angle'] < -20:  # Handling negative angles (extension)
        upper_arm_score = 2
    else:
        upper_arm_score = 1

    # Lower Arm Score
    if 60 <= row['Lower_Arm_Angle'] <= 100:
        lower_arm_score = 1
    else:
        lower_arm_score = 2

    # Wrist Score
    if abs(row['Wrist_Angle']) < 15:
        wrist_score = 1
    else:
        wrist_score = 2

    # Wrist Twist (if available)
    if 'Wrist_Twist' in row:
        if abs(row['Wrist_Twist']) > 15:
            wrist_score += 1

    # Neck Score
    if row['Neck_Angle'] > 45:
        neck_score = 3
    elif row['Neck_Angle'] >= 20:
        neck_score = 2
    elif row['Neck_Angle'] < -10:  # Handling backward neck extension
        neck_score = 2
    else:
        neck_score = 1

    # Trunk Score
    if row['Trunk_Angle'] > 60:
        trunk_score = 4
    elif row['Trunk_Angle'] > 45:
        trunk_score = 3
    elif row['Trunk_Angle'] >= 20:
        trunk_score = 2
    elif row['Trunk_Angle'] < -5:  # Handling trunk extension
        trunk_score = 2
    else:
        trunk_score = 1

        
        
        
        

    # Final RULA Score (simplified version)
    final_rula_score = (
        upper_arm_score +
        lower_arm_score +
        wrist_score +
        neck_score +
        trunk_score
    )

    return pd.Series({
        'UpperArm_Score': upper_arm_score,
        'LowerArm_Score': lower_arm_score,
        'Wrist_Score': wrist_score,
        'Neck_Score': neck_score,
        'Trunk_Score': trunk_score,
        'Final_RULA_Score': final_rula_score
    })

# Apply scoring function
scores_df = angles_df_filtered.apply(rula_scoring, axis=1)

plt.figure(figsize=(15, 5))
plt.plot(data['Time'], scores_df['UpperArm_Score'], linestyle='-', color='b', label='Upper Arm RULA Score')
plt.plot(data['Time'], scores_df['LowerArm_Score'], linestyle='-', color='r', label='Lower Arm RULA Score')
plt.plot(data['Time'], scores_df['Wrist_Score'], linestyle='-', color='g', label='Wrist RULA Score')
plt.plot(data['Time'], scores_df['Neck_Score'], linestyle='-', color='m', label='Neck RULA Score')
plt.plot(data['Time'], scores_df['Trunk_Score'], linestyle='-', color='c', label='Trunk RULA Score')
plt.plot(data['Time'], scores_df['Final_RULA_Score'], linestyle='-', color='y', label='Final RULA Score')
plt.xlabel('Time (s)')
plt.ylabel('RULA Score')
plt.title('RULA Score vs Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
fig, axes = plt.subplots(6, 1, figsize=(9, 8), sharex=True, gridspec_kw={'hspace': 0.2})

scores = ['UpperArm_Score', 'LowerArm_Score', 'Wrist_Score', 'Neck_Score', 'Trunk_Score', 'Final_RULA_Score']
colors = ['b', 'r', 'g', 'm', 'c', 'y']
labels = ['Upper Arm RULA Score', 'Lower Arm RULA Score', 'Wrist RULA Score', 'Neck RULA Score', 'Trunk RULA Score', 'Final RULA Score']

for ax, score, color, label in zip(axes, scores, colors, labels):
    ax.plot(data['Time'], scores_df[score], linestyle='-', color=color, label=label)
    ax.set_ylabel('Score')
    ax.legend(loc='upper right', fontsize='small')
    ax.grid(True)
    ax.set_ylim(min(scores_df[score]) - 1, max(scores_df[score]) + 1)

axes[-1].set_xlabel('Time (s)')
plt.suptitle('RULA Scores - ATSF08 with VR, Trial 3', y=0.91)

#plt.tight_layout()
output_path = "Rula_scores.pdf"  # Specify your desired file path
plt.savefig(output_path, format='pdf',bbox_inches='tight')

plt.show()